<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook works correctly with kernel **`IBM Runtime 23.1 on Python 3.10 XS`** if using IBM Watson Studio or else use standard Python 3.10 runtime. It shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- Setup
- Binding machine learning engine
- Subscriptions
- Performance monitor, scoring and payload logging
- Quality monitor and feedback logging
- Fairness, Drift monitoring and explanations

<a id="setup"></a>
## 1. Setup

### Requirements installation

In [ ]:
# Install following dependencies if you are not running notebook in IBM Watson Studio Env

#!pip install --upgrade "ibm-watson-openscale~=3.0.34" --no-cache | tail -n 1
#!pip install --upgrade boto3 --no-cache | tail -n 1
#!pip install --upgrade pandas --no-cache | tail -n 1

In [ ]:
!pip3 install sagemaker --no-cache | tail -n 1

**Action:** Restart the kernel.

### Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

- Download this [notebook](https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/AWS%20Sagemaker/notebooks/Credit%20%20model%20with%20SageMaker%20linear-learner%20.ipynb) to create SageMaker model
- Run the notebook to train a SageMaker model and create deployment endpoint for online inference

### 1.2 Authentication

Import and initiate.

#### ACTION: Get OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```bash
ibmcloud target -g <myResourceGroup>
```
- get details of the instance
```bash
ibmcloud resource service-instance 'AI-OpenScale-instance_name'
```

In [1]:
CLOUD_API_KEY = '<CLOUD_API_KEY>'

In [2]:

DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [3]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

In [4]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_RESOURCE_CRN="<COS_RESOURCE_CRN>"
COS_API_KEY_ID = "<COS_API_KEY_ID>"
COS_ENDPOINT = "https://s3-api.us-geo.objectstorage.softlayer.net" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints


In [5]:
BUCKET_NAME = "<BUCKET_NAME>" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training_recoded.csv"

In [6]:
!rm credit_risk_training_recoded.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_training_recoded.csv"

--2024-07-11 14:59:09--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_training_recoded.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 639376 (624K) [text/plain]
Saving to: ‘credit_risk_training_recoded.csv’

credit_risk_trainin 100%[===================>] 624.39K  1.88MB/s    in 0.3s    

2024-07-11 14:59:10 (1.88 MB/s) - ‘credit_risk_training_recoded.csv’ saved [639376/639376]



### Store training data in COS for OpenScale reference

In [7]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [8]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

/Users/nelwin/Desktop/-/drift/drift/mynv/lib/python3.10/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.bluemix.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### Initiate OpenScale client

In [9]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(authenticator=authenticator,service_url="https://aiopenscale.cloud.ibm.com")
wos_client.version

'3.0.37'

Create schema for data mart.

In [10]:
wos_client.data_marts.show()

AIOSFASTPATH-80E6093F-5ACF-4EB7-9DA6-7BA9BF56A929,None,True,active,2024-05-16 06:09:07.089000+00:00,80e6093f-5acf-4eb7-9da6-7ba9bf56a929


In [11]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.DB2, # For DB2 use DatabaseType.DB2
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 80e6093f-5acf-4eb7-9da6-7ba9bf56a929


In [12]:
wos_client.data_marts.get(data_mart_id).result.to_dict()

{'metadata': {'id': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:data_mart:80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'url': '/v2/data_marts/80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'created_at': '2024-05-16T06:09:07.089000Z',
  'created_by': 'IBMid-662005298W',
  'modified_at': '2024-05-16T06:09:08.796000Z',
  'modified_by': 'IBMid-662005298W'},
 'entity': {'name': 'AIOSFASTPATH-80E6093F-5ACF-4EB7-9DA6-7BA9BF56A929',
  'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929::',
  'internal_database': True,
  'database_configuration': {'database_type': 'postgresql',
   'credentials': {'secret_id': '3538c3d0-1265-4092-8592-d3a928bbc1d7'},
   'location': {'schema_name': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929'}},
  'status': {'state': 'active'}}}

<a id="binding"></a>
## 2. Bind machine learning engines

### Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [13]:
SAGEMAKER_ENGINE_CREDENTIALS = {
                   'access_key_id': '<ACCESS_KEY_ID>', 
                   'secret_access_key': '<SECRET_ACCESS_KEY>', 
                   'region': '<REGION>'}

In [14]:
SERVICE_PROVIDER_NAME = "AWS Machine Learning"
SERVICE_PROVIDER_DESCRIPTION = "Added by AWS tutorial WOS notebook."

In [15]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [16]:
added_service_provider_result=wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description="AWS Service Provider",
        service_type=ServiceTypes.AMAZON_SAGEMAKER,
        operational_space_id = "production", 
        credentials=SageMakerCredentials(
            access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
            secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'],
            region=SAGEMAKER_ENGINE_CREDENTIALS['region']
        ),
        background_mode=False
    ).result



service_provider_id = added_service_provider_result.metadata.id
print("Service Provider id ", service_provider_id)




 Waiting for end of adding service provider 4fa5625b-5401-4a31-a423-d480ea184cda 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------


Service Provider id  4fa5625b-5401-4a31-a423-d480ea184cda


In [17]:
wos_client.service_providers.show()

None,active,AWS Machine Learning,amazon_sagemaker,2024-07-11 09:29:39.272000+00:00,4fa5625b-5401-4a31-a423-d480ea184cda
None,active,OpenScale Headless Service Provider,custom_machine_learning,2024-07-05 13:55:20.532000+00:00,0bf0f943-24cf-487e-b971-4a6f586effc0
89cbf463-87d9-4032-9c0c-e371dd68d156,active,aWatson Machine Learning V2_amzon,watson_machine_learning,2024-06-06 06:03:03.459000+00:00,84265de0-8f4b-4257-8743-7c2525e86b00
89cbf463-87d9-4032-9c0c-e371dd68d156,active,Watson Machine Learning V2,watson_machine_learning,2024-06-02 08:24:57.362000+00:00,e13bfbbb-e639-4875-bbd5-6d2de8e834d0
89cbf463-87d9-4032-9c0c-e371dd68d156,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2024-05-16 06:09:28.464000+00:00,19fc94cc-9ed2-4ac6-a98e-1acf1a9e729e
89cbf463-87d9-4032-9c0c-e371dd68d156,active,WOS ExpressPath WML production binding,watson_machine_learning,2024-05-16 06:09:12.960000+00:00,7ef8cc5f-65b1-4f83-9d1b-d2d508420958


In [18]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

{'resources': [{'metadata': {'guid': 'cc7acf711b19c689d67e7b2a01b88ca2',
    'url': 'Credit-risk-endpoint-scoring-2021-06-24-05-32',
    'created_at': '2021-06-24T05:40:41.734Z',
    'modified_at': '2023-04-25T07:04:18.041Z'},
   'entity': {'name': 'Credit-risk-endpoint-scoring-2021-06-24-05-32',
    'deployment_rn': 'arn:aws:sagemaker:us-east-2:014862798213:endpoint/credit-risk-endpoint-scoring-2021-06-24-05-32',
    'type': 'online',
    'scoring_endpoint': {'url': 'Credit-risk-endpoint-scoring-2021-06-24-05-32'},
    'asset': {'asset_id': '98783e76b0aedefc06c70bd74fa2356e',
     'asset_rn': 'arn:aws:sagemaker:us-east-2:014862798213:model/credit-risk-linear-learner-2021-06-24-05-32',
     'url': 's3://sagemaker-us-east-2-014862798213/credit-risk/output/Credit-risk-linear-learner-2021-06-24-05-32/output/model.tar.gz',
     'name': 'Credit-risk-linear-learner-2021-06-24-05-32',
     'asset_type': 'model',
     'created_at': '2021-06-24T05:40:27.363Z'},
    'asset_properties': {'asset_r

In [19]:
deployment_id='***' # Get GUID for the deployment created using the other notebook "Credit model with SageMaker linear-learner.ipynb"
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break
model_asset_details

{'metadata': {'guid': 'cc7acf711b19c689d67e7b2a01b88ca2',
  'url': 'Credit-risk-endpoint-scoring-2021-06-24-05-32',
  'created_at': '2021-06-24T05:40:41.734Z',
  'modified_at': '2023-04-25T07:04:18.041Z'},
 'entity': {'name': 'Credit-risk-endpoint-scoring-2021-06-24-05-32',
  'deployment_rn': 'arn:aws:sagemaker:us-east-2:014862798213:endpoint/credit-risk-endpoint-scoring-2021-06-24-05-32',
  'type': 'online',
  'scoring_endpoint': {'url': 'Credit-risk-endpoint-scoring-2021-06-24-05-32'},
  'asset': {'asset_id': '98783e76b0aedefc06c70bd74fa2356e',
   'asset_rn': 'arn:aws:sagemaker:us-east-2:014862798213:model/credit-risk-linear-learner-2021-06-24-05-32',
   'url': 's3://sagemaker-us-east-2-014862798213/credit-risk/output/Credit-risk-linear-learner-2021-06-24-05-32/output/model.tar.gz',
   'name': 'Credit-risk-linear-learner-2021-06-24-05-32',
   'asset_type': 'model',
   'created_at': '2021-06-24T05:40:27.363Z'},
  'asset_properties': {'asset_revision': '1682406258041'}}}

<a id="subsciption"></a>
## 3. Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [20]:
aws_asset = Asset(
        asset_id=model_asset_details['entity']['asset']['asset_id'],
        name=model_asset_details['entity']['asset']['name'],
        url=model_asset_details['entity']['asset']['url'],
        asset_type=model_asset_details['entity']['asset']['asset_type'] if 'asset_type' in model_asset_details['entity']['asset'] else 'model',
        problem_type=ProblemType.BINARY_CLASSIFICATION,
        input_data_type=InputDataType.STRUCTURED,
    )

In [21]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url'] )

In [22]:
deployment = AssetDeploymentRequest(
        deployment_id=model_asset_details['metadata']['guid'],
        url=model_asset_details['metadata']['url'],
        name=model_asset_details['entity']['name'],
        deployment_type=model_asset_details['entity']['type'],
        scoring_endpoint =  scoring_endpoint
    )

In [23]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )

In [24]:
feature_columns = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes', 'LoanDuration', 'LoanAmount', 'InstallmentPercent', 'CurrentResidenceDuration', 'Age', 'ExistingCreditsCount', 'Dependents']
categorical_columns = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes']

In [25]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='predicted_label',
        probability_fields=['score'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=None,
        output_data_schema=None,
        feature_fields=feature_columns,
        categorical_fields=categorical_columns
    )

In [26]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=aws_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription 95e480af-2fec-43ef-b34e-b3a24572b35d 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'95e480af-2fec-43ef-b34e-b3a24572b35d'

#### List subscriptions

In [27]:
wos_client.subscriptions.show()

98783e76b0aedefc06c70bd74fa2356e,model,Credit-risk-linear-learner-2021-06-24-05-32,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,cc7acf711b19c689d67e7b2a01b88ca2,Credit-risk-endpoint-scoring-2021-06-24-05-32,4fa5625b-5401-4a31-a423-d480ea184cda,active,2024-07-11 09:30:10.069000+00:00,95e480af-2fec-43ef-b34e-b3a24572b35d
d4259f6b-e5a1-4a33-a862-d2788e5289ef,model,[asset] GCR Headless Subscription,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,aa8bd973-d680-496b-923d-f9bcbcc0bc7a,[asset] GCR Headless Subscription,0bf0f943-24cf-487e-b971-4a6f586effc0,active,2024-07-05 13:57:01.523000+00:00,345f4646-871a-4e0c-b30f-abcf9a83ea71
ad45394c-0cfb-403c-81f6-a20f44cded7a,model,Spark German Risk Model - Final,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,8db12c5c-94cf-486e-937a-6f17dadf8a56,New - Spark German Risk Deployment - Final,e13bfbbb-e639-4875-bbd5-6d2de8e834d0,active,2024-07-04 09:15:28.269000+00:00,59622413-a37f-4e76-a4d5-46967f3cf12b
43870a88-5fb5-425a-9099-078b0469dd2a,model,Amazon Mobile Review Classifier,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,8edfc920-a4bc-47cd-83fd-9e9674476dec,Amazon Mobile Review Classifier deployment2,84265de0-8f4b-4257-8743-7c2525e86b00,active,2024-06-06 06:04:06.732000+00:00,7828e5f6-9cc7-4cc6-b20b-7bf844c38ccc
699a18fc-1d52-4a1b-9a25-87d9362cd3dc,model,Scikit German Risk Model WML V4,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,2de6645d-c6a9-42ae-a884-4e2bb646ad33,Scikit German Risk Deployment WML V4-1,e13bfbbb-e639-4875-bbd5-6d2de8e834d0,active,2024-06-02 08:26:51.212000+00:00,91a07f4c-50b6-426d-93da-94438a7edc9a
a7fc7ed9-f000-4f27-988a-54fbc8e397e6,model,GermanCreditRiskModelChallenger,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,38059feb-e4c5-4037-90ea-a18f77a8ec5b,GermanCreditRiskModelChallenger,19fc94cc-9ed2-4ac6-a98e-1acf1a9e729e,active,2024-05-16 06:09:55.832000+00:00,81044f9c-27a8-42f2-90be-9cef8b33fe39


<a id="scoring"></a>
## 4. Performance metrics, scoring and payload logging

### Score the credit risk model and measure response time

In [28]:
import requests
import time
import json
import boto3

In [29]:
subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
subscription_details

{'metadata': {'id': '95e480af-2fec-43ef-b34e-b3a24572b35d',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:subscription:95e480af-2fec-43ef-b34e-b3a24572b35d',
  'url': '/v2/subscriptions/95e480af-2fec-43ef-b34e-b3a24572b35d',
  'created_at': '2024-07-11T09:30:10.069000Z',
  'created_by': 'IBMid-662005298W',
  'modified_at': '2024-07-11T09:30:12.124000Z',
  'modified_by': 'IBMid-662005298W'},
 'entity': {'data_mart_id': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'service_provider_id': '4fa5625b-5401-4a31-a423-d480ea184cda',
  'asset': {'asset_id': '98783e76b0aedefc06c70bd74fa2356e',
   'url': 's3://sagemaker-us-east-2-014862798213/credit-risk/output/Credit-risk-linear-learner-2021-06-24-05-32/output/model.tar.gz',
   'name': 'Credit-risk-linear-learner-2021-06-24-05-32',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_reference': 

In [30]:
endpoint_name = subscription_details['entity']['deployment']['name']

payload = "0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,12,4152,2,3,29,2,1"

In [31]:
runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": 0.19936884939670563,
      "predicted_label": 0
    }
  ]
}


### Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with OpenScale standard.

In [32]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  d06ef733-fc03-46a6-9988-89e9446e7760


In [33]:
values = [float(s) for s in payload.split(',')]

request_data = {'fields': feature_columns, 
                'values': values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [34]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, background_mode=False,request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....



 Waiting for end of storing records with request id: c4951a50-b2d1-42b0-9a98-27c581a0c8b4 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


Number of records in the payload logging table: 1


In [35]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

1.0,None,0.0,0.0,0.0,0.0,4152.0,f1406015-8360-42aa-9f2a-42de5bb0b81b-1,0.0,0.0,1.0,2.0,0,0.0,2024-07-11T09:30:42.450Z,1.0,1.0,0.0,0.8006311506032944,0.0,"[0.8006311506032944, 0.19936884939670563]",0.0,0.0,0.19936884939670563,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,29.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,cc7acf711b19c689d67e7b2a01b88ca2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [36]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 5fca0414-5453-44ed-8eee-3b90e3695627 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [37]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'5fca0414-5453-44ed-8eee-3b90e3695627'

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [38]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
feedback_dataset_id

'ebce88fb-722f-4fd2-9ace-f43302b96f1d'

The feedback records can be send to feedback table using below code.

In [39]:
import requests
import pandas as pd
import numpy as np
import time

time.sleep(10) #It gives enough time for dataset creation

data = pd.read_csv('https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_feedback_recoded.csv',header=0,dtype=float)
feedback_columns = data.columns.tolist()
feedback_records = data.values.tolist()

payload_scoring =  [{"fields": feedback_columns, "values": feedback_records}]
wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)




 Waiting for end of storing records with request id: a802af07-7413-4339-82dc-e64e5a87dfa0 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [40]:
wos_client.data_sets.print_records_schema(data_set_id=feedback_dataset_id)

CheckingStatus,string,True
LoanDuration,double,True
CreditHistory,string,True
LoanPurpose,string,True
LoanAmount,double,True
ExistingSavings,string,True
EmploymentDuration,string,True
InstallmentPercent,double,True
Sex,string,True
OthersOnLoan,string,True
CurrentResidenceDuration,double,True


In [41]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

196

In [42]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 52821088-b20b-4756-b67e-a28f0773db97 




finished

---------------------------
 Successfully finished run 
---------------------------




In [43]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-08-07 10:23:22.965000+00:00,true_positive_rate,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.42424242424242425,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,area_under_roc,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.6813519813519814,0.8,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,precision,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.7777777777777778,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,matthews_correlation_coefficient,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.4426747010626324,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,f1_measure,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.5490196078431373,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,accuracy,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.7653061224489796,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,label_skew,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.6909336273400487,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,gini_coefficient,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.36270396270396277,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,log_loss,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.4547315703073752,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 10:23:22.965000+00:00,false_positive_rate,f8d88915-b45f-4a61-bba0-09b9f58bf0f2,0.06153846153846154,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,52821088-b20b-4756-b67e-a28f0773db97,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


Note: First 10 records were displayed.


<a id="datamart"></a>
## 6. Get the logged data

### Payload logging

#### Print schema of payload_logging table

In [44]:
wos_client.data_sets.print_records_schema(data_set_id=payload_data_set_id)

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
CheckingStatus,string,True
LoanDuration,integer,True
CreditHistory,string,True
LoanPurpose,string,True
LoanAmount,integer,True
ExistingSavings,string,True
EmploymentDuration,string,True


<a id="fairness_and_explainability"></a>
## 7. Fairness, Drift monitoring and explanations

### Get payload data

In [45]:
scoring_data_filename='credit_risk_scoring_recoded.csv'
scoring_data_filename_json='credit_risk_scoring_recoded.json'

In [46]:
!rm credit_risk_scoring_recoded.json
!rm credit_risk_scoring_recoded.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.json"
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.csv"

--2024-08-07 15:54:25--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5108 (5.0K) [text/plain]
Saving to: ‘credit_risk_scoring_recoded.json’

credit_risk_scoring 100%[===================>]   4.99K  --.-KB/s    in 0.03s   

2024-08-07 15:54:25 (156 KB/s) - ‘credit_risk_scoring_recoded.json’ saved [5108/5108]

--2024-08-07 15:54:25--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133,

In [47]:
sm_runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

with open(scoring_data_filename) as f_payload:
    scoring_response = sm_runtime.invoke_endpoint(EndpointName = endpoint_name,
                                                  ContentType = 'text/csv',
                                                  Body = f_payload.read().encode())
    
    result = json.loads(scoring_response['Body'].read().decode())
    print(json.dumps(result, indent=2))

In [48]:
f = open(scoring_data_filename_json,"r")
payload_values = json.load(f)
request_data = {'fields': feature_columns, 
                'values': payload_values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}


In [49]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 208


In [50]:
wos_client.data_sets.show_records(payload_data_set_id)

,250,ce94d7419c28c598543294fbc4f5f864-1,1,less_100,2024-08-07T09:59:07.624427Z,0.971782546473715,less_0,car_new,No Risk,skilled,stores,none,4,19,all_credits_paid_back,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,unemployed,"[0.971782546473715, 0.028217453526285]"
,250,ce94d7419c28c598543294fbc4f5f864-10,2,100_to_500,2024-08-07T09:59:07.624427Z,0.9699113373984984,0_to_200,car_new,No Risk,skilled,stores,none,15,19,credits_paid_to_date,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,real_estate,1_to_4,"[0.9699113373984984, 0.030088662601501545]"
,5461,ce94d7419c28c598543294fbc4f5f864-100,4,unknown,2024-08-07T09:59:07.624427Z,0.6003104464290147,no_checking,appliances,No Risk,skilled,none,none,36,43,prior_payments_delayed,4,yes,free,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,yes,male,2,car_other,4_to_7,"[0.6003104464290147, 0.39968955357098535]"
,2885,ce94d7419c28c598543294fbc4f5f864-101,3,less_100,2024-08-07T09:59:07.624427Z,0.8271092517948442,0_to_200,car_used,No Risk,skilled,none,none,11,42,prior_payments_delayed,2,yes,own,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,car_other,less_1,"[0.8271092517948442, 0.17289074820515588]"
,250,ce94d7419c28c598543294fbc4f5f864-102,2,less_100,2024-08-07T09:59:07.624427Z,0.98649525682623,less_0,car_new,No Risk,skilled,none,none,6,19,all_credits_paid_back,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,real_estate,less_1,"[0.98649525682623, 0.01350474317377005]"
,7607,ce94d7419c28c598543294fbc4f5f864-103,4,500_to_1000,2024-08-07T09:59:07.624427Z,0.840687670958204,no_checking,appliances,Risk,management_self-employed,none,co-applicant,29,31,outstanding_credit,3,yes,free,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,car_other,greater_7,"[0.15931232904179604, 0.840687670958204]"
,4513,ce94d7419c28c598543294fbc4f5f864-104,3,less_100,2024-08-07T09:59:07.624427Z,0.9772421560353008,less_0,furniture,No Risk,unskilled,bank,none,13,22,all_credits_paid_back,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,less_1,"[0.9772421560353008, 0.0227578439646992]"
,3596,ce94d7419c28c598543294fbc4f5f864-105,2,500_to_1000,2024-08-07T09:59:07.624427Z,0.9881226863409517,0_to_200,furniture,No Risk,skilled,none,none,26,23,all_credits_paid_back,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,real_estate,1_to_4,"[0.9881226863409517, 0.011877313659048358]"
,250,ce94d7419c28c598543294fbc4f5f864-106,2,less_100,2024-08-07T09:59:07.624427Z,0.8937549052583218,less_0,furniture,No Risk,skilled,none,none,19,29,credits_paid_to_date,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,less_1,"[0.8937549052583218, 0.10624509474167822]"
,250,ce94d7419c28c598543294fbc4f5f864-107,2,100_to_500,2024-08-07T09:59:07.624427Z,0.9710072852398339,less_0,car_new,No Risk,skilled,stores,none,11,21,prior_payments_delayed,1,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,real_estate,1_to_4,"[0.9710072852398339, 0.028992714760166103]"


### Enable and run fairness monitoring

In [51]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex_female",
         "majority": [[0,0]],
         "minority": [[1,1]],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": [0],
    "unfavourable_class": [1],
    "min_records": 30
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation da94306a-8542-4b49-b04d-f27b7ae77ec1 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Run fairness monitor

In [52]:
time.sleep(20)
#Note: When you create fairness monitor, initial run is also created
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-08-07 09:59:24.165078+00:00,fairness_value,c9333fb3-82cd-431d-9eb5-65bd64c9f7bd,82.716,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,da94306a-8542-4b49-b04d-f27b7ae77ec1,aff300d0-f16a-45cc-8201-a00790c1df94,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:59:24.165078+00:00,fairness_value,c9333fb3-82cd-431d-9eb5-65bd64c9f7bd,100.0,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,da94306a-8542-4b49-b04d-f27b7ae77ec1,aff300d0-f16a-45cc-8201-a00790c1df94,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


### Enable and run Drift monitoring

#### Drift requires a trained model to be uploaded manually for AWS. You can train, create and download a drift detection model using template given ( check for Drift detection model generation) [here](https://github.com/IBM/watson-openscale-samples/blob/main/training%20statistics/4.5/training_statistics_notebook.ipynb)

In [53]:
!rm -rf creditrisk_aws_drift_detection_model.tar.gz
!wget -O creditrisk_aws_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz?raw=true 

zsh:1: no matches found: https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz?raw=true


In [54]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_aws_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [55]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [56]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 30,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation a62debeb-03ad-4841-98ff-32a8c7b10c8e 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'a62debeb-03ad-4841-98ff-32a8c7b10c8e'

In [57]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run aff300d0-f16a-45cc-8201-a00790c1df94 




running.
finished

---------------------------
 Successfully finished run 
---------------------------




In [58]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-08-07 09:59:46.400114+00:00,data_drift_magnitude,2080115b-d1d8-4bc5-aa20-56251ca634e5,0.0673076923076923,None,0.1,[],drift,505f727e-fa36-473e-91a5-e7f21cbabfc7,bdbf70f7-e301-497c-a135-c49a4dec9d0b,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


### Enable Explainability and run explanation on sample record

In [59]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitoring run bdbf70f7-e301-497c-a135-c49a4dec9d0b 




finished

---------------------------
 Successfully finished run 
---------------------------




Getting a `transaction_id` to run explanation on

In [60]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

021be1fb-c3a0-49dc-a841-8edd11749f70


In [61]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

,250,ce94d7419c28c598543294fbc4f5f864-1,1,less_100,2024-08-07T09:59:07.624427Z,0.971782546473715,less_0,car_new,No Risk,skilled,stores,none,4,19,all_credits_paid_back,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,unemployed,"[0.971782546473715, 0.028217453526285]"
,250,ce94d7419c28c598543294fbc4f5f864-10,2,100_to_500,2024-08-07T09:59:07.624427Z,0.9699113373984984,0_to_200,car_new,No Risk,skilled,stores,none,15,19,credits_paid_to_date,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,real_estate,1_to_4,"[0.9699113373984984, 0.030088662601501545]"
,5461,ce94d7419c28c598543294fbc4f5f864-100,4,unknown,2024-08-07T09:59:07.624427Z,0.6003104464290147,no_checking,appliances,No Risk,skilled,none,none,36,43,prior_payments_delayed,4,yes,free,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,yes,male,2,car_other,4_to_7,"[0.6003104464290147, 0.39968955357098535]"
,2885,ce94d7419c28c598543294fbc4f5f864-101,3,less_100,2024-08-07T09:59:07.624427Z,0.8271092517948442,0_to_200,car_used,No Risk,skilled,none,none,11,42,prior_payments_delayed,2,yes,own,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,car_other,less_1,"[0.8271092517948442, 0.17289074820515588]"
,250,ce94d7419c28c598543294fbc4f5f864-102,2,less_100,2024-08-07T09:59:07.624427Z,0.98649525682623,less_0,car_new,No Risk,skilled,none,none,6,19,all_credits_paid_back,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,real_estate,less_1,"[0.98649525682623, 0.01350474317377005]"
,7607,ce94d7419c28c598543294fbc4f5f864-103,4,500_to_1000,2024-08-07T09:59:07.624427Z,0.840687670958204,no_checking,appliances,Risk,management_self-employed,none,co-applicant,29,31,outstanding_credit,3,yes,free,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,car_other,greater_7,"[0.15931232904179604, 0.840687670958204]"
,4513,ce94d7419c28c598543294fbc4f5f864-104,3,less_100,2024-08-07T09:59:07.624427Z,0.9772421560353008,less_0,furniture,No Risk,unskilled,bank,none,13,22,all_credits_paid_back,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,less_1,"[0.9772421560353008, 0.0227578439646992]"
,3596,ce94d7419c28c598543294fbc4f5f864-105,2,500_to_1000,2024-08-07T09:59:07.624427Z,0.9881226863409517,0_to_200,furniture,No Risk,skilled,none,none,26,23,all_credits_paid_back,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,real_estate,1_to_4,"[0.9881226863409517, 0.011877313659048358]"
,250,ce94d7419c28c598543294fbc4f5f864-106,2,less_100,2024-08-07T09:59:07.624427Z,0.8937549052583218,less_0,furniture,No Risk,skilled,none,none,19,29,credits_paid_to_date,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,less_1,"[0.8937549052583218, 0.10624509474167822]"
,250,ce94d7419c28c598543294fbc4f5f864-107,2,100_to_500,2024-08-07T09:59:07.624427Z,0.9710072852398339,less_0,car_new,No Risk,skilled,stores,none,11,21,prior_payments_delayed,1,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,real_estate,1_to_4,"[0.9710072852398339, 0.028992714760166103]"


In [62]:
payload_data = wos_client.data_sets.get_list_of_records(limit=5,data_set_id=payload_data_set_id,output_type='pandas').result
scoring_ids=payload_data['scoring_id'].tolist()
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

Running explanations on scoring IDs: ['ce94d7419c28c598543294fbc4f5f864-1', 'ce94d7419c28c598543294fbc4f5f864-10', 'ce94d7419c28c598543294fbc4f5f864-100', 'ce94d7419c28c598543294fbc4f5f864-101', 'ce94d7419c28c598543294fbc4f5f864-102']
{
  "metadata": {
    "explanation_task_ids": [
      "6fa53975-ddd3-446f-99b1-b4156fcb50b9",
      "6caec445-5a2b-429d-ba3e-92847fbc05c9",
      "2dc8dca5-a6a2-4433-b0bc-98eb61e3b559",
      "e7a099b6-ceb1-4864-8536-88497cd8b033",
      "a9e029fa-5fc8-4d98-b974-3d5144674768"
    ],
    "created_by": "1000331001",
    "created_at": "2024-08-07T10:41:59.445949Z"
  }
}


'6fa53975-ddd3-446f-99b1-b4156fcb50b9'

In [67]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id, subscription_id=subscription_id).result.to_dict()

{'metadata': {'explanation_task_id': '6fa53975-ddd3-446f-99b1-b4156fcb50b9',
  'created_by': '1000331001',
  'created_at': '2024-08-07T10:41:59.445949Z'},
 'entity': {'status': {'state': 'in_progress'},
  'asset': {'id': '1b9f22c5-ac03-4017-b40b-27969e429379',
   'name': 'Scikit German Risk Model WML V4 - zLinux',
   'input_data_type': 'structured',
   'problem_type': 'binary',
   'deployment': {'id': 'edc1e06c-7bf9-4522-8893-05a919a35ca0',
    'name': 'Scikit German Risk Deployment WML V4 - zLinux'}},
  'scoring_id': 'ce94d7419c28c598543294fbc4f5f864-1'}}

## Congratulations

You have finished the tutorial for IBM Watson OpenScale and AWS Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit AWS model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

---